In [1]:
import ml
import nlp
import json_io
import pickle
from itertools import chain
from dvs import DictVectorizerPartial
import numpy as np

### Process comments

In [2]:
path = ml.JSON_DIR+"twitter/"
sarcastic_path = path+"sarcastic/"
serious_path = path+"serious/"
source = '-twitter-'
features_path = 'features/'
n=1

In [3]:
json_io.processRandomizeJson(sarcastic=True,
                     json_path=sarcastic_path,
                     features_path=features_path,
                     source=source,
                     n=n,
                     cleanTokens=nlp.cleanTokensTwitter)
json_io.processRandomizeJson(sarcastic=False,
                     json_path=serious_path,
                     features_path=features_path,
                     source=source,
                     n=n,
                     cleanTokens=nlp.cleanTokensTwitter)

File unique.json	time:	0:05:49.941728
Processed 52679 json lines
File unique.json	time:	0:14:57.310423
Processed 163298 json lines


### Load set of features

In [4]:
sarcasticFeats = json_io.loadProcessedFeatures(features_path,
                                       source,
                                       sarcastic=True,
                                       n=n,
                                       random=False)
seriousFeats = json_io.loadProcessedFeatures(features_path,
                                     source,
                                     sarcastic=False,
                                     n=n,
                                     random=False)
features = chain(sarcasticFeats, seriousFeats)

### Train and test, reports results

In [5]:
dvp = DictVectorizerPartial()

In [6]:
(X,y) = ml.split_feat(features, 2)

In [ ]:
(X,y) = ml.flatten(X,y)

In [ ]:
(X,y) = (dvp.partial_fit_transform(X), np.array(list(y)))

In [ ]:
pickle.dump(dvp, open('pickled/-twitter-dvp.pickle', 'wb'))
pickle.dump(y, open('pickled/-twitter-y.pickle', 'wb'))
pickle.dump(X, open('pickled/-twitter-X.pickle', 'wb'))

In [ ]:
X = pickle.load(open('pickled/-twitter-X.pickle', 'rb'))
y = pickle.load(open('pickled/-twitter-y.pickle', 'rb'))

### Train and test, reports results

In [ ]:
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
classifiers = [
    LogisticRegression(n_jobs=-1),
    #SGDClassifier(loss='log'),
    BernoulliNB(),
    MultinomialNB(),
]

In [ ]:
results = []
for reduceamount in [0, 1000000, 500000, 100000, 50000, 25000, 10000, 7500, 5000, 2500, 1500, 1000, 750, 500, 250, 100, 50, 10, 5]:
    print("\n\t\tReduction: "+str(reduceamount))
    for trainsize in [0.01, 0.05, 0.1, 0.2, 0.4, 0.6, 0.8]:
        print("\n\t\tTraining size: "+str(trainsize))
        results.append(
            ml.trainTest(X,
                         y,
                         classifiers=classifiers,
                         reduce=reduceamount,
                         splits=2,
                         trainsize=trainsize,
                         testsize=0.2))
pickle.dump(results, open('pickled/-twitter-trained.pickle', 'wb'))
print(results)